In [4]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# === Load Data ===
# Adjust these file paths as needed.
df0 = pd.read_csv("data/prices_round_3_day_0.csv")
df1 = pd.read_csv("data/prices_round_3_day_1.csv")
df2 = pd.read_csv("data/prices_round_3_day_2.csv")

# Combine the three days
df = pd.concat([df0, df1, df2])

df.reset_index(drop=True, inplace=True)

# === Compute Technical Indicators (Bollinger Bands) ===
# These will be used to generate trading signals.
window = 20  # rolling window for the moving average and standard deviation
df['ma'] = df['mid_price'].rolling(window=window, min_periods=1).mean()
df['std'] = df['mid_price'].rolling(window=window, min_periods=1).std()
# Standard Bollinger Bands: 2 standard deviations above and below the moving average.
df['upper'] = df['ma'] + 2 * df['std']
df['lower'] = df['ma'] - 2 * df['std']

# === Simulate a Simple Bollinger Bands Mean Reversion Strategy ===
# Basic idea:
#   - When price falls below the lower band, the asset is considered oversold → BUY
#   - When price rises above the moving average (or upper band), exit the long (or for a short strategy, 
#     you could reverse the logic)
# For simplicity we only generate long signals here.
cash = 0
position = 0            # number of shares held; assume 0 initially
trade_size = 200        # number of shares to trade per signal
portfolio_values = []
trades = []  # to record trade timestamps and prices

for idx, row in df.iterrows():
    price = row['price']
    ma = row['ma']
    lower = row['lower']
    upper = row['upper']
    
    # --- Trading Logic ---
    # Enter long when price is considered oversold:
    if position == 0 and price < lower:
        # Buy trade: go long trade_size shares.
        position += trade_size
        cash -= trade_size * price
        trades.append((row['timestamp'], 'Buy', trade_size, price))
    
    # Exit long when price reverts (e.g. when it crosses above the moving average)
    elif position > 0 and price > ma:
        # Sell trade: exit the entire long position.
        cash += position * price
        trades.append((row['timestamp'], 'Sell', position, price))
        position = 0
        
    # Track portfolio value over time (cash plus current position value)
    portfolio_value = cash + position * price
    portfolio_values.append(portfolio_value)

df['portfolio'] = portfolio_values

# === Plot the Portfolio Value Over Time ===
fig_portfolio = go.Figure()
fig_portfolio.add_trace(go.Scatter(x=df['timestamp'], y=df['portfolio'], mode='lines', name='Portfolio Value'))
fig_portfolio.update_layout(title="Portfolio Value Over Time",
                            xaxis_title="Time",
                            yaxis_title="Portfolio Value")
fig_portfolio.show()

# === Plot Price, Bollinger Bands, and Trade Signals ===
fig_trades = go.Figure()
fig_trades.add_trace(go.Scatter(x=df['timestamp'], y=df['price'], mode='lines', name='Price'))
fig_trades.add_trace(go.Scatter(x=df['timestamp'], y=df['ma'], mode='lines', name='Moving Average'))
fig_trades.add_trace(go.Scatter(x=df['timestamp'], y=df['upper'], mode='lines', name='Upper Band'))
fig_trades.add_trace(go.Scatter(x=df['timestamp'], y=df['lower'], mode='lines', name='Lower Band'))

# Mark trades on the chart
buy_trades = [t for t in trades if t[1] == 'Buy']
sell_trades = [t for t in trades if t[1] == 'Sell']

if buy_trades:
    buy_times, _, _, buy_prices = zip(*buy_trades)
    fig_trades.add_trace(go.Scatter(x=buy_times, y=buy_prices,
                                    mode='markers', marker=dict(color='green', size=8),
                                    name='Buy'))
if sell_trades:
    sell_times, _, _, sell_prices = zip(*sell_trades)
    fig_trades.add_trace(go.Scatter(x=sell_times, y=sell_prices,
                                    mode='markers', marker=dict(color='red', size=8),
                                    name='Sell'))

fig_trades.update_layout(title="Price with Bollinger Bands and Trade Signals",
                         xaxis_title="Time",
                         yaxis_title="Price")
fig_trades.show()

# === Print Summary of Trades and Final Portfolio Value ===
print(f"Final Portfolio Value: {cash + position * df.iloc[-1]['price']:.2f}")
print("Trades Executed:")
for trade in trades:
    print(trade)


KeyError: 'mid_price'